# Google Calendar API

[Descripción general de la API de Google Calendar  |  Google for Developers](https://developers.google.com/calendar/api/guides/overview?hl=es-419)

Guía rapida obtener credenciales de autenticación y los eventos de un calendario:

In [ ]:
# Dependencies
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [1]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]


def main():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    # now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    now = datetime.datetime.now(datetime.UTC).isoformat().rsplit("+", 1)[0] + "Z"
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])

  except HttpError as error:
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=69722697557-n13nrkrq7t630sthrt70fucdcmn19hsl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51582%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=egQvmqpsqsYXGEHG5FEcVpKlmFLF9I&access_type=offline
Getting the upcoming 10 events
2024-12-10T10:00:00-06:00 Reunion con el equipo


---

## Clase Calendar 
Para interactuar con la API de Google Calendar, se puede utilizar la clase `Calendar` que tiene los siguientes métodos:
- `get_credentials()`: Obtiene las credenciales de autenticación.
- `get_service()`: Obtiene el servicio de Google Calendar.
- `get_id_calendar()`: Obtiene el id de todos los calendarios.
- `get_calendars_info()`: Obtiene la información de todos los calendarios.
- `create_event_body()`: Crea el cuerpo de un evento.
- `create_event()`: Crea un evento (con el cuerpo del evento).
- `get_current_time()`: Obtiene la fecha y hora actual.
- `plus_time()`: Suma un tiempo a una fecha dada.
- `get_events()`: Obtiene los eventos de un calendario.

In [34]:
import os.path
from typing import Any, List, Dict, Optional, Union, Literal
from datetime import datetime, timedelta

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]

UpdateType = Literal['none', 'all', 'externalOnly']

class Calendar():
    @classmethod
    def get_credentials(cls, secrets_path: str):
        creds = None
        if os.path.exists(secrets_path+"token.json"):
            creds = Credentials.from_authorized_user_file(secrets_path+"token.json", SCOPES)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    secrets_path+"credentials.json", SCOPES
                )
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open(secrets_path+"token.json", "w") as token:
                token.write(creds.to_json())
            return creds
        
    @classmethod
    def get_service(cls, secrets_path: str = "../secrets/"):
        creds = cls.get_credentials(secrets_path)
        return build("calendar", "v3", credentials=creds)
    
    @classmethod
    def get_id_calendars(cls, service: Any) -> List[str]:
        calendars = []
        for cal in service.calendarList().list().execute().get('items', []):
            if cal.get('selected', None): # select relevant calendars in google calendar UI
                calendars.append(cal['id'])
        return calendars
    
    @classmethod
    def get_calendars_info(cls, service: Any) -> List[dict]:
        calendars = service.calendarList().list().execute()
        data = []
        for item in calendars.get('items', []):
            if item['accessRole'] != 'reader':
                data.append({'kind': item['kind'], 'id': item['id'], 'timeZone': item['timeZone']})
        return data
    
    @classmethod
    def create_event_body(cls, 
                          summary: str, 
                          start_time: str,
                          end_time: str,
                          timezone: str,
                          location: Optional[str] = None,
                          description: Optional[str] = None,
                          recurrence: Optional[List[str]] = None,
                          attendees: Optional[List[Dict[str, str]]] = None,
                          reminders: Optional[Dict[str, Union[bool, List[Dict[str, Union[str, int]]]]]] = None,
                          conference_data: Optional[Dict[str, dict]] = None 
                        ) -> Dict[str, Any]:
        return {
            "summary": summary,
            "location": location,
            "description": description,
            "start": {"dateTime": start_time, "timeZone": timezone},
            "end": {"dateTime": end_time, "timeZone": timezone},
            "recurrence": recurrence,
            "attendees": attendees,
            "reminders": reminders,
            "conferenceData": conference_data
        }
    
    @classmethod
    def create_event(cls, service: Any,
                     body: Dict, 
                     calendarId: str = 'primary', 
                     conferenceData: int = 0) -> str:
    
        event = service.events().insert(calendarId=calendarId, 
                                        body=body, 
                                        conferenceDataVersion=conferenceData).execute()
        return event.get('htmlLink')
    
    @classmethod
    def get_current_time(cls) -> str:
        return datetime.now().astimezone().replace(microsecond=0).isoformat()

    @classmethod
    def plus_time(cls, current_datetime: str, minutes: int) -> str:
        date_object = datetime.fromisoformat(current_datetime).strftime('%Y-%m-%d %H:%M:%S')
        new_date = datetime.strptime(date_object, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=minutes)
        return new_date.replace(microsecond=0).isoformat()
    
    @classmethod
    def get_events(cls, service: Any, calendarId: str = 'primary', 
                   maxResults: int = 10, min_time: Optional[str] = None, 
                   max_time: Optional[str] = None) -> List[Dict[str, str]]:
        events_data = []
        events_result = service.events().list(calendarId=calendarId,
                                              timeMin=min_time if min_time else cls.get_current_time(),
                                              timeMax=max_time,
                                              maxResults=maxResults,
                                              singleEvents=True,
                                              orderBy='startTime').execute()
        events = events_result.get('items', [])
        for event in events:
            start = event["start"].get("dateTime", event["start"].get("date"))
            events_data.append({'id': event['id'], 'start': start, 'summary': event['summary']})

        return events_data
    
    @classmethod
    def delete_event(cls, service: Any,  eventId: str, calendarId: str = 'primary',
                     sendUpdates: Optional[UpdateType] = 'none') -> None:
        try:
            service.events().delete(calendarId=calendarId, eventId=eventId).execute()
        except HttpError as error:
            print(f"An error occurred: {error}")


In [21]:
service = Calendar.get_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=69722697557-n13nrkrq7t630sthrt70fucdcmn19hsl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53710%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=BnqJf8yScI6ArWLoRxA6yW4XaIaCFe&access_type=offline


In [35]:
Calendar.get_id_calendars(service)
Calendar.get_calendars_info(service)

# now = Calendar.get_current_time()
# event = Calendar.create_event_body(summary="Test Event",
#                                    start_time=now,
#                                    end_time=Calendar.plus_time(now, 30),
#                                    timezone='America/Mexico_City')
# Calendar.create_event(service, event)

Calendar.get_events(service)
Calendar.delete_event(service, 'eventId', sendUpdates='none')

[{'id': 'e6cnmcm85pl2cg3o920aph6u34',
  'start': '2024-10-21T17:06:21-06:00',
  'summary': 'Test Event'},
 {'id': 't3shctfr7eoaff2qqp52a3upqo',
  'start': '2024-12-10T10:00:00-06:00',
  'summary': 'Reunion con el equipo'}]